## CoMD

Classical Molecular Dynamics algorithms and workloads

**Links:**  
[Doxygen Documentation](http://exmatex.github.io/CoMD/doxygen-mpi/index.html)  
[github](https://github.com/exmatex/CoMD)  
[Lab Home Page](http://www.exmatex.org/comd.html)  

### Serial Run
(Haswell) Intel(R) Xeon(R) CPU E5-2699 v3 @ 2.30GHz  
  
CPUTIME: 8.94e+06  
94.2% in `ljForce(SimFlat* SimFlat*)` function within `ljForce.c`  
76.7% Total Time in `for (int iOff=iBox*MAXATOMS,ii=0; ii<nIBox; ii++,iOff++)` loop, line 189  

```c 
|189|       for (int jOff=MAXATOMS*jBox,ij=0; ij<nJBox; ij++,jOff++)  //76.7% Total CPU Time
|190|       {                                                         // 189: 9.9% 
|191|          real_t dr[3];
|192|          int jId = s->atoms->gid[jOff];  
|193|          if (jBox < s->boxes->nLocalBoxes && jId <= iId )           //  5.9%
|194|             continue; // don't double count local-local pairs.
|195|          real_t r2 = 0.0;
|196|          for (int m=0; m<3; m++)
|197|          {
|198|             dr[m] = s->atoms->r[iOff][m]-s->atoms->r[jOff][m];      //  6.4% 
|199| **          r2+=dr[m]*dr[m];                                        // 11.7%  
|200|          }
|201|
|202|          if ( r2 > rCut2) continue;                                 //  6.8%
|203|
|204|          // Important note:
|205|          // from this point on r actually refers to 1.0/r
|206|          r2 = 1.0/r2;
|207| **       real_t r6 = s6 * (r2*r2*r2);                               // 12.8%
|208|          real_t eLocal = r6 * (r6 - 1.0) - eShift;                  //  5.4%
|209|          s->atoms->U[iOff] += 0.5*eLocal;                           //  6.6%  
|210|          s->atoms->U[jOff] += 0.5*eLocal;
|211|
|212|          // calculate energy contribution based on whether
|213|          // the neighbor box is local or remote
|214|          if (jBox < s->boxes->nLocalBoxes)
|215|             ePot += eLocal;
|216|          else
|217|             ePot += 0.5 * eLocal;
|218|
|219|          // different formulation to avoid sqrt computation
|220|          real_t fr = - 4.0*epsilon*r6*r2*(12.0*r6 - 6.0);
|221|          for (int m=0; m<3; m++)
|222|          {
|223|             s->atoms->f[iOff][m] -= dr[m]*fr;
|224|             s->atoms->f[jOff][m] += dr[m]*fr;                         
|225|          }
|226|       } // loop over atoms in jBox
```

 ---



clang-5.0.1  
-std=c99 -DDOUBLE -g -march=native -O3

### CPI
`ljForce() | Loop at 189`: .99 Cycles per Instruction  
`line 199`: .72 Cycles per Instruction   
`line 207`: 1.48 Cycles per Instruction

#### Issue Cycles
`ljForce() | Loop at 189`:  
-- 4.51e+09 Full Issue | 28.5%  Cycles Issuing Max Instructions  
-- 1.60e+07 No Issue | 0.1% Cycles Issuing No Instructions  
-- 1.58e+10 Total Cycles   
`line 199`:  
-- 6.90e+08 Full Issue | 29.7% Cycles Issuing Max Instructions  
-- 2.00e+06 No Issue | 0.1% Cycles Issuing No Instructions  
-- 2.32e+09 Total Cycles  
`line 207`:  
-- 1.67e+09 Full Issue | 66.0% Cycles Issuing Max Instructions  
-- 6.00e+06 No Issue | 0.2% Cycles Issuing No Instructions  
-- 2.53e+09 Total Cycles
#### Retire Cycles
`ljForce() | Loop at 189`:  
-- 4.11e+09 Full Retire | 26.0% Cycles Retiring Max Instructions  
-- 4.32e+09 No Retire | 27.3% Cycles Retiring No Instructions  
-- 1.58e+10 Total Cycles   
`line 199`:  
-- 8.40e+08 Full Retire | 36.2% Cycles Retiring Max Instructions   
-- 5.56e+08 No Retire | 24.0% Cycles Retiring No Instructions  
-- 2.32e+09 Total Cycles   
`line 207`:  
-- 6.58e+08 Full Retire | 26.0% Cycles Retiring Max Instructions  
-- 8.16e+08 No Retire | 32.3% Cycles Retiring No Instructions  
-- 2.53e+09 Total Cycles

### Memory
#### Data Cache 
`ljForce() | Loop at 189`:  
-- 4.60e+07 L1 Data Cache Misses | 0.6% L1 Cache Miss Rate  
-- 1.55e+07 L2 Data Cache Misses | 66.3% L1 Misses Hit L2   
-- 7.12e+09 Load/Store Instructions    
`line 199`:  
-- 2.00e+07 L1 Data Cache Misses | 1.6% L1 Cache Miss Rate  
-- 6.72e+06 L2 Data Cache Misses | 66.4% L1 Misses Hit L2  
-- 1.22e+09 Load/Store Instructions  
`line 207`:  
-- 2.00e+06 L1 Data Cache Misses | 0.2% L1 Cache Miss Rate  
-- 6.78e+05 L2 Data Cache Misses | 66.1% L1 Misses Hit L2  
-- 1.24e+09 Load Store Instructions 

In [31]:
1 - (.678 / 2)

0.661



---

### Memory Density 
1.67e+10 Load Instructions.  
2.88e+09 Store Instructions.   
4.02e+10 Total Cycles.   
(1.67 + .288) / 4.02 = .487  
48.7% Memory Density

---  